# Introduction

*   [I The « powergrid control» problem](#powergrid_control)
    *   [The reality](#reality)
    *   [The fiction (aka our "model")](#fiction)
    *   [Zoom in a substation](#substation)
    *   [An object in a substation](#object)
    *   [Different configurations](#config)
    *   [Note on the topology count](#topo_count)
    *   [Introduction of "operational constraints" in grid2op](#constraints)
    *   [Comparison with "the reality"](#comp_with_real)
    
*   [II Summary of Reinforcement Learning](#rl_sum) 

*   [III Other materials](#other_mat) 

In this notebook we will expose what kind of problems grid2op aims at modeling. 

We will first expose the "powergrid control" problem **modeled by grid2op** (there exist thousands of example of powergrid grid problems, we don't model them all, of course...)

And in the second part we will expose the basics of reinforcement learning.

# I The « powergrid control» problem <a class="anchor" id="powergrid_control"></a>


## The reality <a class="anchor" id="reality"></a>
A powergrid is a really complex man made "objects". In the case of grid2op, a powergrid can be represented as a set of "objects" that are:
- powerline "extremity" end
- powerline "origin" end
- load (usually the size of a small city or of a big industrial firm)
- generators (usually big power plants, such as wind farm, dumps, coal or nuclear powerplants for example)

All these objects are connected together in some places called "substation". In reality a substations looks like this:
![](./img/Taipei_Taiwan_Electrical_Substation.jpg)
(image credit https://commons.wikimedia.org/wiki/File:Taipei_Taiwan_Electrical-Substation-at-Taipei-Zoo-South-Station-01.jpg )

And substations are connected together with so called "powerline" that allow some power to flow from one place to another:
![](./img/Powerline_separators.jpg)
(image credit https://commons.wikimedia.org/wiki/File:Powerline-separators.jpg)

## The fiction (aka our "model") <a class="anchor" id="fiction"></a>
Of course, in this framework, we will not detail every objects you can see on these pictures. For us a powergrid (here relatively simple) will be represented as:
![](./img/powergrid.png)

In the above figure you can see:
- substations are represented as circle, you can see 5 on this figure, labelled 0, 1, ..., 4
- powerline are showed with... line on this plot, you can see there 8 powerlines there
- generators are displayed with small green circle, there are two
- loads are displayed with small orange cirle and there is three on this simple powerline.

## Zoom in a substation <a class="anchor" id="substation"></a>
Now let's zoom in (or at least pretend to) in substation with id 2:
![](./img/powergrid_zoom2.png)
As you can see on the right part (bottom) in fact this substations (as all substations in grid2op) have 2 "busbars" (represented here are vertical black line in the zoomed in part). Note that in all these graphs, powerlines are represented with blue lines.

What does it entails ? It entails that we can "split" a substation into different independant "buses". A "bus" is a fancy word from the power system community meaning that: *if two objects are on the same bus, then there exist a direct electrical path in the substation connecting them*. But wait, how can you split buses ?


## An object in a substation <a class="anchor" id="object"></a>
In fact, for each "object" on a substation, in our modeling you have two switches as showed in the image below:
![](./img/powergrid_zoom3.png)

And you can choose to either connect the object (in the case the powerline with id 1) to either busbars (denoted by "a" and "b" in the figure).

So basically, all the goal of the grid2op platform is to be able to model in particular what is the "best" position of these switches. In all grid2op framework, when we speak about the ***topology*** you can imagine it being the list of the position of each switch in the grid.

## Different configurations <a class="anchor" id="config"></a>
This possibility to change the switches configuration, lead to multiple configuration of the elements per substation. If we take the example of the substation 2 (that we already used above) we can list the number of possible configurations:
![](./img/all_topo.png)

**NB** here we only listed the topologies with all element connected (in grid2op it is possible to disconnect the powerline for example). We also didn't take into account the symmetry of the situation. For example, the *topology* we called "*topo 2: {1,4}; {5,6}*" has been here represented with object 1 and 4 connected to busbar *b* and object 5 and 6 to busbar *a* it would have been rigoursly equivalent to assign 5 and 6 to busbar *b* and 1 and 4 to busbar *a*. What matters here is that objects 1 and 4 are electrically isolated from object 5 and 6 in this substation: there is not electrical path that connects 5 and 4 in this *topology*

In order to be "valid" a topology must also ensure some physical constraints. For example:
- we could decide to connect the 1,4 and 5 to busbar *a* and leave 6 alone on busbar *a*. This is not strictly impossible (and it is not equivalent to disconnect powerline 6). In reality it can serve certain purposes, mostly related to voltages. However, we do not recommend to use this kind of topology.
- having a load alone on a busbar (=without having one powerline connected to the same busbar) is equivalent to disconnect it, and will lead to a *game over*.
- having a generator alone on a busbar (=without having one powerline connected to the same busbar) is also equivalent to disconnect it, it will also lead to a game over.


## Note on the topology count <a class="anchor" id="topo_count"></a>
When trying to modify the topology of the powergrid, you can expect 1) highly non linear effect (that we will cover in the next notebook) and a dramatic increase of the number of actions you can perform at each time step.

We want to emphasize this: action space in power system are really humongous. 

### Different possibilities  per substation
As we covered in the previous subsection (Different configurations) the number of different configuration for a substation having *k* elements is on the order of $2^{k-1}$. The bigger the grid, the highest the number of elements will be connected on each nodes. For example, for the standard IEEE 118 grid used in many power system publications, one single substation counts 17 elements. This lead to $\approx 2^16 = 65.536$ configuration for this substation alone (*i.e* if you only care about finding the exact topology of this substations counting 17 elements, you need to choose among $65.000$ possible actions).

### Modifying all substations
But in theory, you could change not only the topology of $1$ substation but on as many as you want (**in theory**, see section *Introduction of "operational constraints" in grid2op* for more information). So say the first substations counts 11 elements, and the second one 17 elements. 

You can choose among $\approx 2^{11-1}$ actions for the first and among $\approx 2^{16-1}$ actions for the second substations, this lead to $1024 *  65.536 \approx 65.000.000$ actions for two substations. There are only 116 more to consider...

As an example, we sum up in the table below the total number of possible topological actions for some grid you can used in grid2op:


|grid2op env name |Number of substations | number of powerlines| total number of topologies |
|-----------------|:--------------------:|:-------------------:|:--------------------------:|
|rte_case5_example|        5             |           8         |       $31.320$             |
|l2rpn_case14_sandbox|     14            |          20         |    $1.397.519.564$         |
|l2rpn_wcci_2020  |        36            |          59         |      $1.88e+21$*           |
|IEEE case 118    |       118            |         186         |       $3.88e+76$*          |

\* these are approximations of course (and some overflow might have occured during the computation)...

### Connection / disconnection of powerlines
Of course at one time step, it is also possible to change the status of the powerline, meaning connecting or disconnecting them. If there are *N* powerlines, the number of such possible actions is then $2^N$.


### Wrapping up: different number of  "actions"
Finally, concerning the number of different actions, we can wrap it up in this table (be carefull this is provided that you allow at each time steps to reconfigure everything)

|grid2op env name |Number of substations | number of powerlines| total number of topologies | number of powerlines action |
|-----------------|:--------------------:|:-------------------:|:--------------------------:|:----------------------------:
|rte_case5_example|        5             |           8         |        $31.320$            |            256              |
|l2rpn_case14_sandbox|     14            |          20         |     $1.397.519.564$        |         $1.048.576$         |
|l2rpn_wcci_2020  |        36            |          59         |       $1.88e+21$*          |         $5.76e+17$*         |
|IEEE case 118    |       118            |         186         |       $3.88e+76$*          |         $9.81e+55$*         |

\* these are approximations of course (and some overflow might have occured during the computation)...

**NB** This is way more than the number of action of Alphastar which is according to [deepming blog](https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii) on the order of $10^{26}$.

## Introduction of "operational constraints" in grid2op <a class="anchor" id="constraints"></a>
In reality though, we modeled different type of "constraints" that makes this total number of "actions" completely irrelevant. We decided, in the grid2op framework to enforce some operational constraints we decided to enforce some limitation that dramatically reduce the action space above. These limitations can be modified from the `Parameters` class


### Of the number of moves someone can perform at the same time
In reality an operator would not switch off half the powerlines of the grid (this would surely lead to a blackout). 

He would not act on more than one or two substations at the same time (this part is probably more limited by the fact humans cannot focus efficiently on many things at the same time).

So we decided to have 2 attributes of the `Parameters` class:
- `MAX_LINE_STATUS_CHANGED` will limit the number of powerline status you can change in a single action (it is usually set to `1`)
- `MAX_SUB_CHANGED` will limit the number of substation you can change at the same time (it is also usually set to 1)

### Of the frequency on which an object might change
Who would make this person in charge of the powergrid ?
![](./img/night_day.webp)
*image credit*: https://architectrucs.tumblr.com/post/109692684233/de-jour-comme-de-nuit

Well the answer is... Not someone that is in charge of a real grid...

And this has nothing to do with the clothing. Indeed, the more you act on a breaker, the more rapidly you can expect it to age, the more fragile it will become. And you can imagine having to change the breakers of extra high voltage part of the powergrid every few months. If you don't imagine the problem it may cause, there is an image of a circuit breaker:
![](./img/circuit_breaker.png)
*image credit*: https://www.globalspec.com/learnmore/electrical_electronic_components/electrical_distribution_protection_equipment/circuit_breakers

So long introduction to say, there are 2 attributes of the `Parameters` class that are present to limit the frequency on which you can act on the same object:
- `NB_TIMESTEP_COOLDOWN_LINE` tells you how many timesteps you have to wait before redoing an action of type connection / disconnection of powerline (it is usually `3` when not `0`)
- `NB_TIMESTEP_COOLDOWN_SUB` indicates how many timesteps you have to wait before making an action on the same substation again (it is usually `3` too).

**NB** these "cooldown" time also enforce some phyiscal constraints. For example, there exist\* elements in real powergrid that cannot be opened / closed more than once every 5 or 10 mins.

### Total number of actions
This introduction is.. already pretty big. So let's summarize the total number of actions that are feasible in grid2op taken into account the limitations above mentionned:

|grid2op env name |Number of substations | number of powerlines| total number of topologies | number of powerlines action |
|-----------------|:--------------------:|:-------------------:|:--------------------------:|:----------------------------:
|rte_case5_example|        5             |           8         |            117             |              8              |
|l2rpn_case14_sandbox|     14            |          20         |            179             |             20              |
|l2rpn_wcci_2020  |        36            |          59         |         $66.811$           |             59              |
|IEEE case 118    |       118            |         186         |         $72.150$           |            186              |

In particular, if your are able to both act on the powerlines and on the substation, the total number of action you might be able to do is  $72.150 * 186 = 13.419.900$ which is already a pretty decent size for an action space (in comparison in go the action space counts 361 actions at most)

## Comparison with "the reality" <a class="anchor" id="comp_with_real"></a>

We want to emphasize that this modeling is an (over) over simplification of the reality. 

- In reality there can also be "switches" that can connect the two busbars (reconfiguring the topology of the substation can be done with only one switch, but on the other hand, sometimes changing one switch will have no effect at all).

- You can also have more than 2 busbars in each substation (sometimes 5 or 6 for example). This makes the number of possible topologies even higher than what it is in grid2op.

- Finally, most of the time a single busbar count a "switch" in its middle that allows to disconnect part of the element connected to it to another part. Basically this entails that some combinaison of elements are not possible to perform

And of course, we model explicitly in this framework (*eg* we allow the agents to act on) only some elements of a powergrid. In reality, much more heterogeneous objects exists with more complex properties. 

We decided to make all these assumptions because we thought it was the easiest setting that allow to perform some topological reconfiguration, beside connecting / disconnecting powerlines.

So keep in mind that the tables presented above are an overly underestimated number of actions companies in charge of the powergrid can take at the same time, mostly because the "real" grid can count more than $10.000$ substations, because much more objects are connected at each substations and also because some substations can often be "split" into a dozen independant "buses". 

# II Summary of Reinforcement Learning <a class="anchor" id="rl_sum"></a>

Though the `Grid2Op` package can be used to perform many different tasks, this set of notebooks will be focused on the machine learning part, and its usage in a Reinforcement learning framework. 

Reinforcement learning is a framework that allows to train an "agent" to solve some task in a time-dependant domain. We tried to cast the grid operation planning task into this framework. The package `Grid2Op` was inspired by it.

In reinforcement learning (RL), there are 2 distinct entities:
* **Environment**: is a modeling of the "world" in which the *agent* takes some *actions* to achieve some pre-definite objectives.
* **Agent**: will take actions on the environment that will have consequences.

These 2 entities exchange 3 main types of information:
* **Action**: is an information sent by the Agent that will modify the internal state of the environment.
* **State** / **Observation**: is the (partial) view of the environment as seen by the Agent. The Agent receives a new state after each action it takes. It can use the observation (state) at time step *t* to take an action at time *t*.
* **Reward**: is the score received by the agent for its previous action. It is the reward that define the objectives of the agent (the goal of the agent is to maximize the reward it receives over time).

A schematic representaiton of this is shown in the figure below (Credit: [Sutton & Barto](http://incompleteideas.net/book/bookdraft2017nov5.pdf)):

![title](img/reinforcement-learning.jpg)

For more information about the problem, please visit the [Example_5bus](Example_5bus.ipynb) notebook which dives more into the modelization of the real-time grid operation planning into a RL framework. Note that this notebook is still under development at the moment.

# III Other materials <a class="anchor" id="other_mat"></a>
Some good materials are also provided in the white paper [Reinforcement Learning for Electricity Network Operation](https://arxiv.org/abs/2003.07339) presented for the L2RPN 2020 Neurips edition.